In [ ]:
import torch
import os
import ignite
import matplotlib.pyplot as plt
from tqdm import tqdm
import pytorch_ssim
from ignite.metrics import SSIM, PSNR
import glob
import numpy as np
from monai.networks.nets import UNet
import monai
from monai import transforms
from monai.utils import set_determinism, first
from monai.transforms import(
    AsDiscrete,
    AddChanneld,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    LoadImaged,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandSpatialCropSamplesd,
    RandShiftIntensityd,
    RandZoomd,
    ScaleIntensityd,
    Spacingd,
    SpatialPadd,
    GaussianSmoothd,
    RandRotate90d,
    ToTensord,
    RandSpatialCropd,
    RandGaussianSmoothd,    
    RandGaussianSharpend,
    RandGaussianNoise,
    
)
from monai.data import (
    DataLoader,
    Dataset,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)

import torch
import torchvision
import numpy as np
from skimage.metrics import structural_similarity as ssim
from tqdm import tqdm

from skimage.metrics import peak_signal_noise_ratio as psnr
from monai.inferers import sliding_window_inference


In [ ]:
!nvidia-smi

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print(device)

In [ ]:
from models.network_swinir import SwinIR as net

model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(64, 128, 256, 512),
        strides=(2, 2, 2),
        num_res_units=3,
    ).to(device)

# model = net(upscale=1, in_chans=1, img_size=192, window_size=8,
#                     img_range=1., depths=[4, 4, 4, 4], embed_dim=90, num_heads=[5, 5, 5, 5],
#                     mlp_ratio=2, upsampler='', resi_connection='1conv').to(device)
model.load_state_dict(torch.load(os.path.join("./weight/UNet_ISMRM_v3.pth")))




In [ ]:
# data_dir
# data_dir2 = glob.glob('/nfs/hufsaims/kjh_shared/pvs_0726/EP_infant_MR_data_curated/*/nifti_3D_iso/T2_dlseg_v0221.nii.gz')
# print(len(data_dir),len(data_dir2)) #둘다 잘 있음 굿 ㅎㅎ

In [ ]:
origin_path =  glob.glob('/nfs/hufsaims/kjh_shared/euna/ISMRM/24mo/*/3D_T2.nii.gz')
low_path = glob.glob('/nfs/hufsaims/kjh_shared/euna/ISMRM/24mo/*/low_3d_T2.nii.gz')

valid_ind = np.arange(40, 47)
test_dict = [
{
    'image' : low_path[i], # low
    'original' : origin_path[i], # original
}for i in valid_ind
    ] 
val_transforms = transforms.Compose(
    [
          transforms.LoadImaged(keys=["image",'original']),
        transforms.EnsureChannelFirstd(keys=["image",'original']),
#         transforms.Spacingd(keys=['image'], pixdim = (1, .5, .5),mode = 'bilinear'),
        transforms.ScaleIntensityd(
                keys=["image",'original'],
                minv=0.0,
                maxv=1.0,
        ),           
#         transforms.SpatialPadd(keys=['image'],spatial_size=(192, 192, 192)),
#         transforms.CenterSpatialCropd(keys=['image'],roi_size = (216, 256, 256)),
        transforms.CenterSpatialCropd(keys=['image','original'],roi_size = (192, 220, 192)),
    ]
)
test_ds = CacheDataset(data=test_dict, transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, num_workers=4)

In [ ]:
improved = nib.load('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/improved/syn_T2_5_6.nii.gz')
improved = improved.get_fdata()

low = nib.load('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/low/low_T2_5.nii.gz')
low = low.get_fdata()
origin = nib.load('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/origin/T2_5.nii.gz')
origin = origin.get_fdata()

unet = nib.load('/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/improved/syn_T2_5_6.nii.gz')
unet = unet.get_fdata()


In [ ]:
plt.figure(dpi=150)
plt.subplot(141)
plt.imshow(origin[:,106,:],cmap='gray')
plt.axis('off')
plt.subplot(142)
plt.imshow(low[:,106,:],cmap='gray')
plt.axis('off')
plt.subplot(143)
plt.imshow(improved[:,106,:],cmap='gray')
plt.axis('off')
plt.subplot(144)
plt.imshow(unet[:,106,:],cmap='gray')
plt.axis('off')




In [ ]:
with torch.no_grad():
    for step, batch in tqdm(enumerate(test_loader)):
        images = batch['image']
        origin = batch['original']
        
#         new = np.zeros((images.shape[2], images.shape[3], zmax-zmin+1))
        print(images.shape)
#         break
        images = images.squeeze(1)
        
    
        plt.figure(dpi = 200)
        plt.subplot(1,4,1)
        plt.imshow(np.rot90(images[0, :,170,:]),cmap = 'gray')
        plt.subplot(1,4,2)
        plt.imshow(np.rot90(images[0,:, 100,:]),cmap = 'gray')
        plt.subplot(1,4,3)

        plt.imshow(np.rot90(images[0,:,80,:]),cmap = 'gray')
        plt.subplot(1,4,4)

        plt.imshow(np.rot90(images[0,:,0,:]),cmap = 'gray')
        plt.show()

In [ ]:
!nvidia-smi

In [ ]:
print(val_inputs.shape)
print(val_outputs.shape)


In [ ]:
!nvidia-smi

In [ ]:
import nibabel as nib

model.eval()
psnr_value = []
ssim_value = []


#bcp
with torch.no_grad():
    for step, batch in tqdm(enumerate(test_loader)):
        zmin = 0
        zmax= 219
        
        file_name=low_path[step].split('/')[-2]
       
        images = batch['image'] #1, 1, 192, 220, 192
        origins = batch['original']#1, 1, 192, 220, 192
        info = nib.load(batch['image_meta_dict']['filename_or_obj'][0]) #300, 320, 208
        
        
        low = np.zeros((images.shape[2], zmax-zmin+1, images.shape[4]))  
        origin = np.zeros((images.shape[2], zmax-zmin+1, images.shape[4]))
        output = np.zeros((images.shape[2],zmax-zmin+1,images.shape[4]))
        
#         images = images.squeeze(0)
#         origins = origins.squeeze(0)
# #         break
        for idx, i in enumerate(range(zmin, 220)):
            val_inputs =images[...,i,:].to(device)
            orig_crop =origins[...,i,:]
#             val_outputs = sliding_window_inference(
#                 val_inputs, [192, 192], 4, model, overlap=0.99,mode='gaussian')
            val_outputs = model(val_inputs)
            plt.figure(dpi = 200)
            plt.subplot(121)
            plt.imshow(val_inputs[0,0,...].detach().cpu().numpy(),cmap='gray')
            plt.axis('off')
            plt.subplot(122)
            plt.imshow(val_outputs[0,0,...].detach().cpu().numpy(),cmap='gray')
            plt.axis('off')
            plt.show()

            
            output[:,idx,:] = val_outputs[0,0].detach().cpu().numpy()
            origin[:,idx,:] = orig_crop[0,0]
            low[:,idx,:] = val_inputs[0,0].detach().cpu().numpy()

            
        
        
#         low_res_nifti = nib.Nifti1Image(low,info.affine,info.header)
#         nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/low/low_T2_{step}.nii.gz') 
        
#         originals = nib.Nifti1Image(origin,info.affine,info.header)
#         nib.save(originals,f'/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/SwinIR/origin/T2_{step}.nii.gz') 
        
        output = nib.Nifti1Image(output,info.affine,info.header)
        nib.save(output,f'/nfs/hufsaims/kjh_shared/euna/ISMRM/Result/UNet/improved/syn_T2_{step}_6.nii.gz')  
        
#         low_res_nifti = nib.Nifti1Image(new1, af)
#         nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/result/0902/input_T2_{step}.nii.gz')   

        

In [ ]:
len(test_loader)

In [ ]:
import nibabel as nib
x = 22
with torch.no_grad():
    for step, batch in tqdm(enumerate(test_loader)):
        zmin = 50
        zmax= 150
        if step == 1 or step == 3:
            zmin = 0
            zmax= 100
        images = batch['image']
        origins = batch['original']#1, 1, 216, 220, 208
        bgs = batch['bgmask']#1, 1, 216, 220, 208
        info = nib.load(batch['image_meta_dict']['filename_or_obj'][0])
        
        low = np.zeros((images.shape[2], zmax-zmin+1, images.shape[4]))  
        origin = np.zeros((images.shape[2], zmax-zmin+1, images.shape[4]))
        bg = np.zeros((images.shape[2],zmax-zmin+1,images.shape[4]))
        output = np.zeros((images.shape[2],zmax-zmin+1,images.shape[4]))
        
#         images = images.squeeze(0)
#         origins = origins.squeeze(0)
#         bgs = bgs.squeeze(0)
#         break
        for idx, i in enumerate(range(zmin, zmax)):
            val_inputs =images[...,i,:].to(device)
            bg_crop =bgs[...,i,:]
            orig_crop =origins[...,i,:]

#             val_outputs = sliding_window_inference(
#                 val_inputs, [192, 192], 4, model, overlap=0.99,mode='gaussian')
            val_outputs = model(val_inputs)
            plt.figure(dpi = 200)
            plt.subplot(131)
            plt.imshow(orig_crop[0,0,...].detach().cpu().numpy(),cmap='gray')
            plt.axis('off')
            plt.subplot(132)
            plt.imshow(val_inputs[0,0,...].detach().cpu().numpy(),cmap='gray')
        
            plt.axis('off')
            plt.subplot(133)
            plt.imshow(val_outputs[0,0,...].detach().cpu().numpy(),cmap='gray')
            plt.axis('off')
            plt.show()
        
            
#             print(val_outputs[0,0].shape)
#             print(val_inputs[0,3].shape)
#             print(new1[:,0,:].shape)
            
            output[:,idx,:] = val_outputs[0,0].detach().cpu().numpy()
            origin[:,idx,:] = orig_crop[0,0]
            bg[:,idx,:] = bg_crop[0,0]
#             print(origins.shape,bgs.shape)
            low[:,idx,:] = val_inputs[0,0].detach().cpu().numpy()
#             print(bg_crop.shape)
            
#             plt.imshow(bg_crop[0,0])
#             plt.show()
            
        
    
#         low_res_nifti = nib.Nifti1Image(low,info.affine,info.header)
#         nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/segmentation_swinIR/low_T2/low_T2_{x}.nii.gz') 
        
#         originals = nib.Nifti1Image(origin,info.affine,info.header)
#         nib.save(originals,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/segmentation_swinIR/T2/T2_{x}.nii.gz') 
        
# #         bg = nib.Nifti1Image(bg,info.affine,info.header)
# #         nib.save(bg,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/segmentation_swinIR/bgmask/mask_{x}.nii.gz')    
        
#         output = nib.Nifti1Image(output,info.affine,info.header)
#         nib.save(output,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/segmentation_swinIR/syn_T2/syn_T2_{x}.nii.gz') 
#         x += 1
#         low_res_nifti = nib.Nifti1Image(new1, af)
#         nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/result/0902/input_T2_{step}.nii.gz')   

        

In [ ]:
 low_res_nifti = nib.Nifti1Image(low,info.affine,info.header)
        nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/segmentation_unet/low/low_T2_{step}.nii.gz') 
        
        originals = nib.Nifti1Image(origin,info.affine,info.header)
        nib.save(originals,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/segmentation_unet/T2/T2_{step}.nii.gz') 
        
        bg = nib.Nifti1Image(bg,info.affine,info.header)
        nib.save(bg,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/segmentation_unet/bgmask/mask_{step}.nii.gz')    
        
        output = nib.Nifti1Image(output,info.affine,info.header)
        nib.save(output,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/segmentation_unet/T2_syn/syn_T2_{step}.nii.gz')  


In [ ]:
import nibabel as nib

model.eval()
psnr_value = []
ssim_value = []


zmin = 110
zmax = 170

with torch.no_grad():
    for step, batch in tqdm(enumerate(test_loader)):
        images = batch['image']#[1,1,216,220,220]
        new1 = np.zeros((images.shape[2], images.shape[3], zmax-zmin+1))
        new2 = np.zeros((images.shape[2], images.shape[3], zmax-zmin+1))
        images = images.squeeze(1)
#         break
        for idx, i in enumerate(range(zmin, zmax)):
            val_inputs =images[...,i-3:i+4].permute(0,3,1,2).to(device)
            val_outputs = sliding_window_inference(
                val_inputs, [192, 192], 4, model, overlap=0.95,mode='gaussian')
#             val_outputs = model(val_inputs)
            plt.figure(dpi = 200)
            plt.subplot(121)
            plt.imshow(np.rot90(val_inputs[0,3,...].detach().cpu().numpy()),cmap='gray')
            plt.axis('off')
            plt.subplot(122)
            plt.imshow(np.rot90(val_outputs[0,0,...].detach().cpu().numpy()),cmap='gray')
            plt.axis('off')
            plt.show()
            
            new1[:, :, idx] = val_outputs[0, 0].detach().cpu().numpy()
            new2[:, :, idx] = val_inputs[0, 3].detach().cpu().numpy()
            
        
        af = np.eye(4, 4)
        
#         low_res_nifti = nib.Nifti1Image(new2, af)
#         nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/result_0821/T2_{step}.nii.gz')   
#         low_res_nifti = nib.Nifti1Image(new1, af)
#         nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/result/result_0821/T2_{step}.nii.gz')   

        break


In [ ]:
# NIfTI 이미지 파일 경로 설정
nifti_path = f'/nfs/hufsaims/kjh_shared/result/result_0821/T2_{step}.nii.gz'
# NIfTI 이미지 불러오기
nifti_image = nib.load(nifti_path)
nifti_data = nifti_image.get_fdata()

# 데이터의 최솟값과 최댓값을 이용하여 대비 조정
min_value = np.min(nifti_data)
max_value = np.max(nifti_data)

# 대비 조정된 이미지 데이터
contrast_normalized_data = (nifti_data - min_value) / (max_value - min_value)
min_value = np.min(contrast_normalized_data)
max_value = np.max(contrast_normalized_data)
print(min_value,max_value)

# # 대비 조정된 데이터를 다시 NIfTI 이미지로 저장
# normalized_nifti_image = nib.Nifti1Image(contrast_normalized_data, nifti_image.affine)
# nib.save(normalized_nifti_image, "test.nii.gz")


In [ ]:
# NIfTI 이미지 파일 경로 설정
nifti_path = f'/nfs/hufsaims/kjh_shared/result/result_0821/T2_0_1.nii.gz'
# NIfTI 이미지 불러오기
nifti_image = nib.load(nifti_path)
nifti_data = nifti_image.get_fdata()

# 데이터의 최솟값과 최댓값을 이용하여 대비 조정
min_value = np.min(nifti_data)
max_value = np.max(nifti_data)

# 대비 조정된 이미지 데이터
contrast_normalized_data = (nifti_data - min_value) / (max_value - min_value)
print(min_value,max_value)

# # 대비 조정된 데이터를 다시 NIfTI 이미지로 저장
# normalized_nifti_image = nib.Nifti1Image(contrast_normalized_data, nifti_image.affine)
# nib.save(normalized_nifti_image, "test.nii.gz")

In [ ]:
new1 = np.zeros((images.shape[2], images.shape[3], zmax-zmin+1))

new1.shape

In [ ]:
print(images.shape)
images = images.squeeze(1)
print(images.shape)
val_inputs =images[...,i-3:i+4].permute(0,3,1,2).to(device)
plt.imshow(np.rot90(val_inputs[0,6,...].detach().cpu().numpy()),cmap='gray')


In [ ]:
low_res_nifti = nib.Nifti1Image(new1, af)
nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/result/0823/T2_{step}_v2.nii.gz')   


In [ ]:
low_res_nifti = nib.Nifti1Image(new2, af)
nib.save(low_res_nifti,f'/nfs/hufsaims/kjh_shared/euna/data_bcp/result/0823/input_T2_{step}_v2.nii.gz') 